# WSG-84 GPS data to local ENU
- Calculates ENU coordinates for "Raw Data.cvs" file in same folder
- Raw data should come from PhyPhox app GPS-measurement

In [ ]:
import math
import numpy as np
import pandas as pd

In [ ]:
def geodetic_to_ecef(lat, lon, h):
    # These functions is based on sbarratt geo.py
    # https://gist.github.com/sbarratt/a72bede917b482826192bf34f9ff5d0b
    # (lat, lon) in WSG-84 degrees
    # h in meters
    a = 6378137
    b = 6356752.3142
    f = (a - b) / a
    e_sq = f * (2-f)
    
    lamb = math.radians(lat)
    phi = math.radians(lon)
    s = math.sin(lamb)
    N = a / math.sqrt(1 - e_sq * s * s)

    sin_lambda = math.sin(lamb)
    cos_lambda = math.cos(lamb)
    sin_phi = math.sin(phi)
    cos_phi = math.cos(phi)

    x = (h + N) * cos_lambda * cos_phi
    y = (h + N) * cos_lambda * sin_phi
    z = (h + (1 - e_sq) * N) * sin_lambda

    return x, y, z

def ecef_to_enu(x, y, z, lat0, lon0, h0):
    a = 6378137
    b = 6356752.3142
    f = (a - b) / a
    e_sq = f * (2-f)
        
    lamb = math.radians(lat0)
    phi = math.radians(lon0)
    s = math.sin(lamb)
    N = a / math.sqrt(1 - e_sq * s * s)

    sin_lambda = math.sin(lamb)
    cos_lambda = math.cos(lamb)
    sin_phi = math.sin(phi)
    cos_phi = math.cos(phi)

    x0 = (h0 + N) * cos_lambda * cos_phi
    y0 = (h0 + N) * cos_lambda * sin_phi
    z0 = (h0 + (1 - e_sq) * N) * sin_lambda

    xd = x - x0
    yd = y - y0
    zd = z - z0

    xEast = -sin_phi * xd + cos_phi * yd
    yNorth = -cos_phi * sin_lambda * xd - sin_lambda * sin_phi * yd + cos_lambda * zd
    zUp = cos_lambda * cos_phi * xd + cos_lambda * sin_phi * yd + sin_lambda * zd

    return xEast, yNorth, zUp

def geodetic_to_enu(lat, lon, h, lat_ref, lon_ref, h_ref):
    x, y, z = geodetic_to_ecef(lat, lon, h)
    
    return ecef_to_enu(x, y, z, lat_ref, lon_ref, h_ref)

In [ ]:
df = pd.read_csv("Raw Data.csv",sep="\t",decimal=",")
df = df.drop(['Altitude (m)','Direction (°)','Satellites'],axis=1)

df["East"] = np.nan
df["North"] = np.nan
df["Up"] = np.nan

ref = df.mean(axis=0)
lat_ref = ref[1]
lon_ref = ref[2]
h_ref = ref[4]

for i, row in df.iterrows():
    lat = row['Latitude (°)']
    lon = row['Longitude (°)']
    h = row['Altitude WGS84 (m)']
    row["East"],row["North"],row["Up"] = geodetic_to_enu(lat,lon,h,lat_ref,lon_ref,h_ref)

df.to_csv('GPSdata.csv')